<a href="https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Advanced%20TensorFlow/Extending%20Keras/Week%201%20-%20Functional%20API/exercise-answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.



# Multiple Output Models using Keras Functional API

Use keras functional API to train a network to predict two different outputs. For this example you will use the **[Wine Quality Dataset](https://archive.ics.uci.edu/ml/datasets/Wine+Quality)** from the **UCI machine learning repository**. It has separate datasets for red wine and white wine.

Normally the wines are classified into one of the quality ratings specified in the attributes. In our example you will combine the two datasets so to predict the wine quality and whether the wine is red or white solely from the attributes. 

You will model wine quality estimations as a regression problem and wine type detection as a binary classification problem.


## Imports

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
  
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools




## Load Dataset
Here is where you can download the data for the wines. 

You will add a new column named 'is_red' in our dataframe to indicate if the wine is white or red. 

In the white wine datset you will fill the column 'is_red' with  zeros and in the red wine dataset you will fill it with ones.


In [ ]:
URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv'
white_df = pd.read_csv(URL, sep=";")
white_df["is_red"] = 0
white_df = white_df.drop_duplicates(keep='first')


In [ ]:
print(white_df.alcohol[0])
print(white_df.alcohol[100])
#EXPECTED OUTPUT
#8.8
#9.1

In [ ]:
URL = 'http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
red_df = pd.read_csv(URL, sep=";")
red_df["is_red"] = 1
red_df = red_df.drop_duplicates(keep='first')


In [ ]:
print(red_df.alcohol[0])
print(red_df.alcohol[100])
#EXPECTED OUTPUT
#9.4
#10.2

Next, the two datasets will be concatenated.

In [ ]:
df = pd.concat([red_df, white_df], ignore_index=True)


In [ ]:
print(df.alcohol[0])
print(df.alcohol[100])
#EXPECTED OUTPUT
#9.4
#9.5

In [ ]:
# NOTE: In a real-world scenario you should shuffle the data
# YOU ARE NOT going to do that here, because we want to test
# with deterministic data. But if you want the code to do it
# it's in the commented line below:

#df = df.iloc[np.random.permutation(len(df))]

This will chart the quality of the wines

In [ ]:
df['quality'].hist(bins=20) 

We can see from the plot above that wine quality dataset is imbalanced. Since there are very few observations with quality 3, 4, 8 and 9, you can drop these observations from our dataset. You can do this by removing data belonging all classes except those >4 and <8

In [ ]:
df = df[(df['quality'] > 4) & (df['quality'] < 8 )]
df = df.reset_index(drop=True)

In [ ]:
print(df.alcohol[0])
print(df.alcohol[100])
#EXPECTED OUTPUT
#9.4
#10.9

You can plot again to see the new range of data and quality

In [ ]:
df['quality'].hist(bins=20) 

Next you can split the datasets into training, test and validation datasets.

The data frame should be split 80:20 into train and test

The resulting train should then be split 80:20 into train and validation


In [ ]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)

In [ ]:
print(train.size)
print(test.size)
print(val.size)

#EXPECTED OUTPUT
# 41015
# 12831
# 10257

Here's where you can explore the training stats. You can pop the labels 'is_red' and 'quality' from the data as these will be used as the labels


In [ ]:
train_stats = train.describe()
train_stats.pop('is_red')
train_stats.pop('quality')
train_stats = train_stats.transpose()

Explore the training stats!

In [ ]:
train_stats

Next pop the quality and type columns from the dataframe and convert it into numpy arrays. Do this for the train, validation and test datasets.



In [ ]:
def format_output(data):
  is_red = data.pop('is_red')
  is_red = np.array(is_red)
  quality = data.pop('quality')
  quality = np.array(quality)
  return (quality, is_red)

In [ ]:
train_Y = format_output(train)
val_Y = format_output(val)
test_Y = format_output(test)


Next, you can normalize the data using the formula:
**X - mean(X) / Std(X)**

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

In [ ]:
norm_train_X = norm(train)
norm_val_X = norm(val)
norm_test_X = norm(test)

## Defining the Model

Define the model using the functional API. The base model will simply be 2 dense layers of 128 neurons each.

In [ ]:
def base_model(inputs):
  x = tf.keras.layers.Dense(units = '128', activation = 'relu')(inputs)
  x = tf.keras.layers.Dense(units = '128', activation = 'relu')(x)
  return x
  

The final model will need two outputs. 

Add output layers to the base model for wine quality output. 

The first, for wine quality will be a dense layer with 1 neuron that will be used for regression.

The other, for wine type will be a dense layer with 1 neuron that will be used for classification (and thus activated by sigmoid)

In [ ]:
def final_model(inputs):
  x = base_model(inputs)
  
  wine_quality = Dense('1', name = 'wine_quality')(x)
  wine_type = Dense(units = '1', activation = 'sigmoid', name = 'wine_type')(x)
  
  model = Model(inputs = inputs, outputs = [wine_quality, wine_type])
  
  return model

## Compiling the Model

Next, compile the model. Use different loss functions for the two outputs. 

Since you will performing binary classification on wine type, you should use the binary crossentropy loss function for it.

As wine quality is a regression, you should use mean squared error as its loss function.

Similarly, you should also specify the metrics for two different outputs. Use RMSE  for wine quality and accuracy for wine type

In [ ]:
inputs = tf.keras.layers.Input(shape=(11,))
rms = tf.keras.optimizers.RMSprop(lr=0.0001)
model = final_model(inputs)
model.compile(optimizer=rms, 
              loss = {'wine_type' : 'binary_crossentropy',
                      'wine_quality' : 'mse'
                     },
              metrics = {'wine_type' : 'accuracy',
                         'wine_quality': tf.keras.metrics.RootMeanSquaredError()
                       }
             )

## Training the Model

Fit the model to the training inputs and outputs


In [ ]:
history = model.fit(norm_train_X, train_Y,
                    epochs = 180, validation_data=(norm_val_X, val_Y))

Gather the training metrics

In [ ]:
loss, wine_quality_loss, wine_type_loss, wine_quality_rmse, wine_type_accuracy = model.evaluate(x=norm_val_X, y=val_Y)



In [ ]:
print(loss)
print(wine_quality_loss)
print(wine_type_loss)
print(wine_quality_rmse)
print(wine_type_accuracy)
# EXPECTED VALUES
# ~ 0.34 - 0.38
# ~ 0.34 - 0.38
# ~ 0.018 - 0.022
# ~ 0.56 - 0.62
# ~ 0.97 - 1.0
# Example:
#0.3657050132751465
#0.3463745415210724
#0.019330406561493874
#0.5885359048843384
#0.9974651336669922

## Analyze the Model Performance

Note that the model has two outputs. The output at index 0 is quality and index 1 is wine type

So, round the quality predictions to the nearest integer.

In [ ]:
predictions = model.predict(norm_test_X)
quality_pred = predictions[0]
type_pred = predictions[1]

In [ ]:
print(quality_pred[0])
# EXPECTED OUTPUT
# 5.6 - 6.0

In [ ]:
print(type_pred[0])
print(type_pred[946])
# EXPECTED OUTPUT
# A number close to zero
# A number close to or equal to 1

### Plot Utilities

In [ ]:
def plot_metrics(metric_name, title, ylim=5):
  plt.title(title)
  plt.ylim(0,ylim)
  plt.plot(history.history[metric_name],color='blue',label=metric_name)
  plt.plot(history.history['val_' + metric_name],color='green',label='val_' + metric_name)


In [ ]:
def plot_confusion_matrix(y_true, y_pred, title='', labels=[0,1]):
  cm = confusion_matrix(y_true, y_pred)
  fig = plt.figure()
  ax = fig.add_subplot(111)
  cax = ax.matshow(cm)
  plt.title('Confusion matrix of the classifier')
  fig.colorbar(cax)
  ax.set_xticklabels([''] + labels)
  ax.set_yticklabels([''] + labels)
  plt.xlabel('Predicted')
  plt.ylabel('True')
  fmt = 'd'
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
          plt.text(j, i, format(cm[i, j], fmt),
                  horizontalalignment="center",
                  color="black" if cm[i, j] > thresh else "white")
  plt.show()

In [ ]:
def plot_diff(y_true, y_pred, title = '' ):
  plt.scatter(y_true, y_pred)
  plt.title(title)
  plt.xlabel('True Values')
  plt.ylabel('Predictions')
  plt.axis('equal')
  plt.axis('square')
  plt.plot([-100, 100], [-100, 100])
  return plt

### Plots for Metrics

In [ ]:
plot_metrics('wine_quality_root_mean_squared_error', 'RMSE', ylim=2)

In [ ]:
plot_metrics('wine_type_loss', 'Wine Type Loss', ylim=0.2)

### Plots for Confusion Matrix

Plot the confusion matrices for wine type. You can see that the model performs well for prediction of wine type from the confusion matrix and the loss metrics.

In [ ]:
plot_confusion_matrix(test_Y[1], np.round(type_pred), title='Wine Type', labels = [0, 1])

In [ ]:
scatter_plot = plot_diff(test_Y[0], quality_pred, title='Type')